# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
import numpy as np
import pandas as pd

import re

import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')



In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

## Подготовка

In [ ]:
try:
    comments = pd.read_csv('/toxic_comments.csv') 
except:
    comments = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
display(comments.head(5))

print('-----------------------------------------------------------------')
comments.info()

print('-----------------------------------------------------------------')

print('Дубликатов -', comments.duplicated().sum())

print('-----------------------------------------------------------------')

print('Пропусков:')
display(comments.isna().sum())

print('-----------------------------------------------------------------')

print('Соотношение в целевом признаке:')
display(comments.toxic.value_counts(normalize=True))


<div class="alert alert-block alert-info">  
Выводы
<li>В таблице 159 292 объектa. Пропусков нет, явных дубликатов нет</li>
<li>Тексты комментариев на английском, есть лишние знаки типа "\nMore\n</li>
<li>Наблюдаем явный дизбаланс классов</li>
<li>Необходимо избавиться от столбца Unnamed, так как он фактически дублирует индексы</li>
</div>

In [ ]:
#функция для очищения текстов постов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [ ]:
#очищаю тексты:
comments['text'] = comments['text'].apply(clear_text) 

In [ ]:
comments.head()

In [ ]:
#ввожу функцию РОS-тэгирования слов:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#ввожу функцию леммализации тектов постов:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [ ]:
#леммализирую тексты постов:
comments['text'] = comments['text'].apply(lemm_text) 

In [ ]:
comments.head()

In [ ]:
#разделяю выборки в соотношении 60/20/20:
features = comments.drop(['toxic'], axis=1) 
target = comments.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.4, 
                                                                              random_state=34)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=.5,
                                                                            random_state=34)

In [ ]:
#смотрю размеры выборок:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)

In [ ]:
#смотрю соотношение 1/0 в выборках на примере target_train:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

print('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

In [ ]:
#уменьшаю кол-во 0 в выборках train:

comments_train = comments.iloc[target_train.index]
target_train_0 = comments_train[comments_train['toxic'] == 0]['toxic']
target_train_1 = comments_train[comments_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = comments.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=42)

features_train_resample = features_train_resample.text 

print('Соотношение 1/0 в тренировочной выборке:')
print(target_train_resample.value_counts(normalize=True))
print()
print(features_train_resample.shape)
print(target_train_resample.shape)


<div class="alert alert-block alert-info">  
Выводы
<li>Удалил ненужный столбец</li>
<li>Очистил тексты комментариев от ненужных знаков, леммализировал, убрал стоп-слова</li>
<li>Сбалансировал данные в целевом признаке</li>
</div>


## Обучение



<div class="alert alert-block alert-info">  
Логистическая регрессия
</div>

In [ ]:
features_train = features_train.text

In [ ]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([42]),
              'lr__max_iter': ([200])}

gscv_log = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_log.fit(features_train, target_train)

mts = gscv_log.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv_log.best_params_)
print()

#валидация:
predictions_valid = gscv_log.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))


<div class="alert alert-block alert-info">  
Дерево решений
</div>

In [ ]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([42]), 
              'dtc__class_weight': (['balanced'])}

gscv_tree = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_tree.fit(features_train_resample, target_train_resample)

mts = gscv_tree.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv_tree.best_params_)
print()

#валидация:
predictions_valid = gscv_tree.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))



<div class="alert alert-block alert-info">  
CatBoostClassifier
</div>

In [ ]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),
              'cbc__class_weights':([(1, 1), (1, 11)])} #вот вообще не уверена, что class_weights тут нужен

gscv_cat = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_cat.fit(features_train_resample, target_train_resample)

mts = gscv_cat.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv_cat.best_params_)
print()

#валидация:
predictions_valid = gscv_cat.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))




<div class="alert alert-block alert-info">  
RandomForestClassifier
</div>

In [ ]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("rfc", RandomForestClassifier())])
    
parameters = {'rfc__n_estimators': ([x for x in range(10, 30)]),
              'rfc__random_state': ([42]),
              'rfc__max_depth': ([x for x in range(1, 10)]),
              'rfc__criterion': (['entropy']),
              'rfc__class_weight': (['balanced'])}

gscv_rfc = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_rfc.fit(features_train_resample, target_train_resample)

mts = gscv_rfc.cv_results_['mean_test_score']
rfc_train_f1 = max(mts)

print('F1 случайного леса =', round(rfc_train_f1,2))
print('при параметрах', gscv_rfc.best_params_)
print()

#валидация:
predictions_valid = gscv_rfc.predict(features_valid.text)
rfc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 случайного леса на валидации =', round(rfc_valid_f1,2))




<div class="alert alert-block alert-info">  
SGDClassifier

</div>

In [ ]:
#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("clf", SGDClassifier())])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5),
              'clf__random_state': ([42]),
              'clf__class_weight': (['balanced'])}

gscv_sgd = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_sgd.fit(features_train_resample, target_train_resample)

mts = gscv_sgd.cv_results_['mean_test_score']
sgdc_train_f1 = max(mts)

print('F1 SGDClassifier =', round(sgdc_train_f1,2))
print('при параметрах', gscv_sgd.best_params_)
print()

#валидация:
predictions_valid = gscv_sgd.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
print('F1 SGDClassifier на валидации =', round(sgdc_valid_f1,2))

In [ ]:

index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'RandomForestClassifier',
         'SGDClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    rfc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        rfc_valid_f1,
                                        sgdc_valid_f1]}
       

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на валидационной выборке', ascending=False)




<div class="alert alert-block alert-info">  
Тестрирование лучшей модели по результататам ваолидационной выборки LogisticRegression с показателем метрики f1 0.781859

</div>

In [ ]:
predictions_test = gscv_log.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
print('финальный F1 логистической регрессии =', round(lr_test_f1,2))

# Вывод


<div class="alert alert-block alert-info">  
В проекте:

<li>загрузил данные и провел их предобработку - удаление лишних данных, очистку текстов, лемматизацию</li>
<li>обучил 4 модели с разными гиперпараметрами и выбарл лучшую для тестрирования</li>
<li>Провел тестрирование на LogisticRegression итоговый показатель метрики получился 0.76</li>
</div>


<div class="alert alert-block alert-info">  
Аутсайдером среди моделей стали RandomForestClassifier и DecisionTreeClassifier, так как дали наименьшее F1.

</div>


<div class="alert alert-block alert-info">  
Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.76. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75, рекомендовать могу LogisticRegression
</div>